<a href="https://colab.research.google.com/github/sunilgandipadala/NLP-Projects/blob/main/Sentiment_Analysis_Blockoffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SCRAPE THE DATA, using scrapy.... 
we had done scrapy in local editor(VS Code).. Now lets implement here...

In [ ]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 4.0 MB 70.1 MB/s 
     |████████████████████████████████| 93 kB 2.3 MB/s 
     |████████████████████████████████| 3.1 MB 31.3 MB/s 
     |████████████████████████████████| 254 kB 62.9 MB/s 
     |████████████████████████████████| 57 kB 4.3 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=adc035dc43df48580bfbf8f9e9318bef81f38197aa3c022b1765dd8c664cf5d1
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher


#Scrapy project created as Blackoffer

In [ ]:
!scrapy startproject Blackoffer

New Scrapy project 'Blackoffer', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/Blackoffer

You can start your first spider with:
    cd Blackoffer
    scrapy genspider example example.com


#change directory

In [ ]:
import os
os.chdir('/content/Blackoffer/Blackoffer/spiders')

In [ ]:
%%writefile -a article_text.py
from turtle import title
import scrapy
import pandas as pd
import string

class ArticleSpider(scrapy.Spider):
    
    name = 'Articles'
    allowed_domains = ['insights.blackcoffer.com']
    data = pd.read_excel("/content/Input.xlsx")
    start_urls= list(data["URL"])

    def parse(self, response):
        title =  response.xpath("//h1/text()").get()
        url = response.url
        url_name =""
        for i in url:
            if i in string.punctuation:
                url_name +="_"
            if i.isalnum():
                url_name += i
        text = response.xpath("//div[@class='td-post-content']/p/text()").getall()
        file = open("{}.txt".format(url_name),"w",encoding='cp1252')#need to create them as url names.
        file.write(title)
        file.write("\n")
        for t in text:
            file.write(t)
            print("------*------")
        file.close()
        yield{
            title:title
        }


Writing article_text.py


THIS WILL GENERATE THE FILES... with ARTICLE TEXT.

In [ ]:
!scrapy crawl Articles

2022-10-28 19:17:54 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: Blackoffer)
2022-10-28 19:17:54 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.7.15 (default, Oct 12 2022, 19:14:55) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
2022-10-28 19:17:54 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'Blackoffer',
 'NEWSPIDER_MODULE': 'Blackoffer.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['Blackoffer.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-10-28 19:17:54 [asyncio] DEBUG: Using selector: EpollSelector
2022-10-28 19:17:54 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-10-28 19:17:54 [scrapy.utils.log] DEBUG: Using asyncio event loop: asy

# Import the required libraries

In [ ]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import string,re,csv

In [ ]:
nlp = spacy.load('en_core_web_sm')
stopwords = list(STOP_WORDS)
punct = string.punctuation
punct=punct+"\n"

#Data Reading


In [ ]:
input_urls = pd.read_excel(r'/content/Output Data Structure.xlsx')
urls = list(input_urls['URL'])

In [ ]:
file_names = []
for url in urls:
  url_name =""
  for i in url:
    if i in string.punctuation:
      url_name +="_"
    if i.isalnum():
      url_name += i
  file_names.append(url_name)
print(len(file_names))

114


In [ ]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("es") or word.endswith("ed"):
        count = 0
    if count == 0:
        count += 1
    return count

#Data Cleaning to Data Analysis

In [ ]:
Total_outputs =[]
for file_name in file_names:
  data = []
  try:
    with open("{}.txt".format(file_name),"r",encoding ='cp1252') as da:
      data.append(da.readlines())
    title = data[0][0]
    sentences = data[0][1].split(".")
    sentences.insert(0,title)
  except:
    continue
  #Tokenization
  data_tokens=[]
  for i in sentences:
    doc = nlp(i)
    for token in doc:
      data_tokens.append(token.text)
  #cleaning data
  data_cleaned = [word for word in data_tokens if word not in stopwords and word not in punct]
  total_words =len(data_cleaned)
  total_sens = len(sentences)
  #Extracting declared variables
  positive_words = open("/content/positive-words.txt","r")
  positive=list(positive_words)
  negative_words = open("/content/negative-words.txt","r",encoding ="cp1252")
  negative= list(negative_words)
  
  pos_words = [word for word in data_cleaned if word+"\n" in positive ]
  neg_words = [word for word in data_cleaned if word+"\n" in negative ]

  pos_score = len(pos_words)
  neg_score = len(neg_words)
  polarity_score = (pos_score -neg_score)/((pos_score+neg_score)+0.000001)
  subjectivity_score = (pos_score+neg_score)/total_words

  #Analysis of Readability
  #Syllable Count
  syllable_count_words = {}
  for word in data_cleaned:
    syllable_count_words[word] = syllable_count(word)
  #Complex words
  complex_words = []
  for key, value in syllable_count_words.items():
    if value>2:
      complex_words.append(key)
  com_word_count = len(complex_words)
  personal_pron =0
  propRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
  for sent in sentences:
    pronouns = propRegex.findall(sent)
    personal_pron += len(pronouns)

  avg_word_sents = avg_sl = total_words/total_sens
  per_com_words = com_word_count/total_words
  fog_index = 0.4 *(avg_sl + per_com_words)

  sum_count_characters = 0
  for sent in sentences:
    new_sen = sent.replace(" ","")
    sum_count_characters+= len(new_sen)
  avg_word_len = sum_count_characters/total_words

  outputs = (pos_score,neg_score,polarity_score,subjectivity_score,avg_sl,per_com_words,fog_index,avg_word_sents,com_word_count,total_words,len(syllable_count_words),personal_pron,avg_word_len)
  Total_outputs.append(outputs)

#Write the data into the file..
ok do one thing...

initially read all files at once and find all the output variables of each link.. and save them as a list in a main list ..

in the end write code for every file...

In [ ]:
table = pd.read_excel(r'/content/Output Data Structure.xlsx')
columns = list(table.columns)[2:]
for j in range(len(Total_outputs)):
  for i in range(len(columns)):
    col = columns[i]
    table.loc[j, col]=Total_outputs[j][i]

#this will save to csv file..
table.to_csv('Output Data Structre.csv')

# To save the data in output file format.

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
writer = pd.ExcelWriter(r'/content/Output Data Structre.xlsx')
table.to_excel(writer)
# save the excel
writer.save()